In [1]:
### You will need mlens package in addition to the libraries you are familiar with
## pip install mlens

In [2]:
!python -m pip install xgboost
!python -m pip install mlens

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
from mlens.ensemble import SuperLearner
from mlens.model_selection import Evaluator
from mlens.metrics import make_scorer
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.svm import SVC
import sklearn
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

[MLENS] backend: threading


### WEEK 9 LAB: 
1. LOOP THROUGH MODELS AS DONE IN THIS WEEK'S CLASS.


In [4]:
# for clf, label in zip([LR, RF, KNC, DTC, ABC, BC, GBC, XGB],
#                       ['Logistic Regression',
#                        'Random Forest',
#                        'KNeighbors',
#                        'Decision Tree',
#                        'Ada Boost',
#                        'Bagging',
#                        'Gradient Boosting',
#                        'XGBoost']):

In [5]:
col_names = [
    "Age", "WorkClass", "fnlwgt", "Education", "EducationNum",
    "MaritalStatus", "Occupation", "Relationship", "Race", "Gender",
    "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "Income"]

df = pd.read_csv('adult.data')
df.columns=col_names
df['Income'] = df['Income'].apply(lambda x: 0 if x == ' <=50K' else 1)

In [6]:
for column in df.columns:
    if df[column].dtype == type(object):
        le = sklearn.preprocessing.LabelEncoder()
        df[column] = le.fit_transform(df[column])
df.tail()

,Age,WorkClass,fnlwgt,Education,EducationNum,MaritalStatus,Occupation,Relationship,Race,Gender,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,Income
32555,27,4,257302,7,12,2,13,5,4,0,0,0,38,39,0
32556,40,4,154374,11,9,2,7,0,4,1,0,0,40,39,1
32557,58,4,151910,11,9,6,1,4,4,0,0,0,40,39,0
32558,22,4,201490,11,9,4,1,3,4,1,0,0,20,39,0
32559,52,5,287927,11,9,2,4,5,4,0,15024,0,40,39,1


In [7]:
X = df.drop(columns='Income')
y = df['Income']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=555, stratify=y)

In [8]:
LR = LogisticRegression(solver='lbfgs', max_iter=10000, random_state=555)
RF = RandomForestClassifier(n_estimators = 100, random_state=555)
SVM = SVC(random_state=0, probability=True)
KNC = KNeighborsClassifier()
DTC = DecisionTreeClassifier()
ABC = AdaBoostClassifier(n_estimators = 100)
BC = BaggingClassifier(n_estimators = 100)
GBC = GradientBoostingClassifier(n_estimators = 100)
# clf_XGB = XGBClassifier(n_estimators = 100, objective= 'binary:logistic', seed=555, use_label_encoder=False)
clf_XGB = XGBClassifier(n_estimators = 100, seed=555, use_label_encoder=False, eval_metric='logloss')
clfs = []
print('5-fold cross validation:\n')
for clf, label in zip([LR, RF, KNC, DTC, ABC, BC, GBC, clf_XGB],
                      ['Logistic Regression',
                       'Random Forest',
                       #'Support Vector Machine',
                       'KNeighbors',
                       'Decision Tree',
                       'Ada Boost',
                       'Bagging',
                       'Gradient Boosting',
                       'XGBoost']):
    scores = sklearn.model_selection.cross_val_score(clf, X_train, y_train, cv=5, scoring="accuracy")
    print("Train CV Accuracy: %0.3f (+/- %0.3f) [%s]" % (scores.mean(), scores.std(), label))
    md = clf.fit(X_train, y_train)
    clfs.append(md)
    print("Test Accuracy: %0.4f " % (sklearn.metrics.accuracy_score(clf.predict(X_test), y_test)))

5-fold cross validation:

Train CV Accuracy: 0.791 (+/- 0.006) [Logistic Regression]
Test Accuracy: 0.8034 
Train CV Accuracy: 0.856 (+/- 0.004) [Random Forest]
Test Accuracy: 0.8575 
Train CV Accuracy: 0.772 (+/- 0.002) [KNeighbors]
Test Accuracy: 0.7792 
Train CV Accuracy: 0.810 (+/- 0.005) [Decision Tree]
Test Accuracy: 0.8145 
Train CV Accuracy: 0.864 (+/- 0.002) [Ada Boost]
Test Accuracy: 0.8636 
Train CV Accuracy: 0.849 (+/- 0.002) [Bagging]
Test Accuracy: 0.8576 
Train CV Accuracy: 0.864 (+/- 0.002) [Gradient Boosting]
Test Accuracy: 0.8650 
Train CV Accuracy: 0.866 (+/- 0.002) [XGBoost]
Test Accuracy: 0.8709 


2. BUILD A CUSTOM ENSEMBLE (SUPERLEARNER) WITH ANY THREE OF THE MODELS IN TWO LAYERS.  IS IT BETTER?


In [9]:
# --- Build ---
# Passing a scoring function will create cv scores during fitting
# the scorer should be a simple function accepting to vectors and returning a scalar
ensemble = SuperLearner(scorer=accuracy_score, random_state=555, verbose=2)

In [10]:
# Build the first layer
ensemble.add([KNC, ABC, DTC])

SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=4782, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...4F10F70>)],
   n_jobs=-1, name='group-0', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=555, sample_size=20,
       scorer=<function accuracy_score at 0x0000024934F10F70>,
       shuffle=False, verbose=2)

In [11]:
# Attach the final meta estimator
ensemble.add_meta(LogisticRegression())
#ensemble.add_meta(GradientBoostingClassifier())

SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=4782, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...4F10F70>)],
   n_jobs=-1, name='group-1', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=555, sample_size=20,
       scorer=<function accuracy_score at 0x0000024934F10F70>,
       shuffle=False, verbose=2)

In [12]:
# Fit ensemble
ensemble.fit(X_train, y_train)


Fitting 2 layers
Processing layer-1             done | 00:00:06
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:06


SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=4782, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...4F10F70>)],
   n_jobs=-1, name='group-1', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=555, sample_size=20,
       scorer=<function accuracy_score at 0x0000024934F10F70>,
       shuffle=False, verbose=2)

In [13]:
#pred_vals = ensemble.predict(X_test)
print ("Accuracy - Train : ", sklearn.metrics.accuracy_score(ensemble.predict(X_train), y_train))
print ("Accuracy - Test : ", sklearn.metrics.accuracy_score(ensemble.predict(X_test), y_test))


Predicting 2 layers
Processing layer-1             done | 00:00:01
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:02
Accuracy - Train :  0.8667485667485667

Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:00
Accuracy - Test :  0.8636363636363636


In [41]:
print("Fit data:\n%r" % ensemble.data)

Fit data:
                                   score-m  score-s  ft-m  ft-s  pt-m  pt-s
layer-1  adaboostclassifier           0.86     0.00  3.62  0.00  0.64  0.17
layer-1  decisiontreeclassifier       0.81     0.00  0.13  0.00  0.00  0.00
layer-1  kneighborsclassifier         0.84     0.00  0.31  0.03  4.92  0.53



In [14]:
'We see the superlearner accuracy is higher than that of all the models in the 5-fold cross validation except of XGboost. XGboost test accuracy is 0.871 while the superlearner one is 0.864.'

'We see the superlearner accuracy is higher than that of all the models in the 5-fold cross validation except of XGboost. XGboost test accuracy is 0.871 while the superlearner one is 0.864.'

3. NOW ADD A PREPROCESSING METHOD OF YOUR CHOICE TO THE DATA BEFORE SEDNING TO YOUR CUSTOM ENSEMBLE FROM #2.  SHOW BOTH RESULTS.

In [15]:
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
ensemble = SuperLearner(scorer=accuracy_score, random_state=555, verbose=2)

In [17]:
# Build the first layer
ensemble.add([KNC, ABC, DTC])

SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=4782, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...4F10F70>)],
   n_jobs=-1, name='group-2', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=555, sample_size=20,
       scorer=<function accuracy_score at 0x0000024934F10F70>,
       shuffle=False, verbose=2)

In [18]:
# Attach the final meta estimator
ensemble.add_meta(LogisticRegression())
#ensemble.add_meta(GradientBoostingClassifier())

SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=4782, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...4F10F70>)],
   n_jobs=-1, name='group-3', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=555, sample_size=20,
       scorer=<function accuracy_score at 0x0000024934F10F70>,
       shuffle=False, verbose=2)

In [19]:
# Fit ensemble
ensemble.fit(X_train, y_train)


Fitting 2 layers
Processing layer-1             done | 00:00:09
Processing layer-2             done | 00:00:00
Fit complete                        | 00:00:09


SuperLearner(array_check=None, backend=None, folds=2,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=4782, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=2, raise_on_ex...4F10F70>)],
   n_jobs=-1, name='group-3', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=555, sample_size=20,
       scorer=<function accuracy_score at 0x0000024934F10F70>,
       shuffle=False, verbose=2)

In [20]:
#pred_vals = ensemble.predict(X_test)
print ("Accuracy - Train : ", sklearn.metrics.accuracy_score(ensemble.predict(X_train), y_train))
print ("Accuracy - Test : ", sklearn.metrics.accuracy_score(ensemble.predict(X_test), y_test))


Predicting 2 layers
Processing layer-1             done | 00:00:13
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:13
Accuracy - Train :  0.8911957411957412

Predicting 2 layers
Processing layer-1             done | 00:00:04
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:04
Accuracy - Test :  0.8628992628992629


In [21]:
print("Fit data:\n%r" % ensemble.data)

Fit data:
                                   score-m  score-s  ft-m  ft-s  pt-m  pt-s
layer-1  adaboostclassifier           0.86     0.00  4.61  0.02  0.68  0.22
layer-1  decisiontreeclassifier       0.81     0.00  0.14  0.02  0.01  0.01
layer-1  kneighborsclassifier         0.84     0.00  0.37  0.04  4.99  0.76

